In [10]:
# Paso 1.
# Preprocese los datos.
# - Cree la columna 'Age' a partir de la columna 'Year'.
#   Asuma que el año actual es 2021.
# - Elimine las columnas 'Year' y 'Car_Name'.

In [11]:
import pandas as pd

data_train = pd.read_csv('../files/input/train_data.csv.zip', index_col = False, compression = "zip")
data_test = pd.read_csv("../files/input/test_data.csv.zip", index_col = False, compression = "zip")

In [12]:
data_train['Age'] = 2021 - data_train['Year']
data_test['Age'] = 2021 - data_test['Year']

data_train.drop(columns=['Year', 'Car_Name'], inplace=True)
data_test.drop(columns=['Year', 'Car_Name'], inplace=True)

In [13]:
# Paso 2.
# Divida los datasets en x_train, y_train, x_test, y_test.

In [14]:
x_train = data_train.drop(columns=['Present_Price'])
y_train = data_train["Present_Price"]
x_test = data_test.drop(columns=['Present_Price'])
y_test = data_test["Present_Price"]

In [15]:
# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Escala las variables numéricas al intervalo [0, 1].
# - Selecciona las K mejores entradas.
# - Ajusta un modelo de regresion lineal.

In [16]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression

categorical_features = ['Fuel_Type', 'Selling_type', 'Transmission']
numerical_features = list(set(x_test.columns) - set(categorical_features))

# Paso 3: Transformador para combinar variables categóricas y numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', MinMaxScaler(), numerical_features)
    ], remainder = StandardScaler()
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),         
    ('feature_selection', SelectKBest(f_regression)), 
    ('regre', LinearRegression()),  
])

In [17]:
# Paso 4.
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use el error medio absoluto
# para medir el desempeño modelo.

In [18]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "feature_selection__k": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
    #'regre__fit_intercept': [True, False],
    #'regre__positive' : [True, False],
    #'regre__copy_X' : [True, False]
}

model = GridSearchCV(
    pipeline, param_grid, cv=10, scoring='neg_mean_absolute_error', n_jobs=-1)

model.fit(x_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder=StandardScaler(),
                                                          transformers=[('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Fuel_Type',
                                                                          'Selling_type',
                                                                          'Transmission']),
                                                                        ('num',
                                                                         MinMaxScaler(),
                                                                         ['Owner',
                                                                          'Age',
                                                                          'Driven_kms',
                                                                          'Selling_Price'])])),
                                       ('feature_selection',
                                        SelectKBest(score_func=<function f_regression at 0x000001C2FF90B700>)),
                                       ('regre', LinearRegression())]),
             n_jobs=-1,
             param_grid={'feature_selection__k': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                                                  11, 12, 13]},
             scoring='neg_mean_absolute_error')

In [19]:
# Paso 5.
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
# Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.

In [20]:
import os
import pickle
import gzip

dir_path = '../files/models'

if not os.path.exists(dir_path):
    os.makedirs(dir_path)
    with gzip.open('../files/models/model.pkl.gz', 'wb') as f:
        pickle.dump(model, f)
else:
    with gzip.open('../files/models/model.pkl.gz', 'wb') as f:
        pickle.dump(model, f)

In [21]:
# Paso 6.
# Calcule las metricas r2, error cuadratico medio, y error absoluto medio
# para los conjuntos de entrenamiento y prueba. Guardelas en el archivo
# files/output/metrics.json. Cada fila del archivo es un diccionario con
# las metricas de un modelo. Este diccionario tiene un campo para indicar
# si es el conjunto de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'metrics', 'dataset': 'train', 'r2': 0.8, 'mse': 0.7, 'mad': 0.9}
# {'type': 'metrics', 'dataset': 'test', 'r2': 0.7, 'mse': 0.6, 'mad': 0.8}

In [22]:
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score
import json

y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

train_metrics = {
    "type": "metrics",
    'dataset': 'train',
    'r2': float(r2_score(y_train, y_train_pred)),
    'mse': float(mean_squared_error(y_train, y_train_pred)),
    'mad': float(median_absolute_error(y_train, y_train_pred)),
}

test_metrics = {
    "type": "metrics", 
    'dataset': 'test',
    'r2': float(r2_score(y_test, y_test_pred)),
    'mse': float(mean_squared_error(y_test, y_test_pred)),
    'mad': float(median_absolute_error(y_test, y_test_pred)),
}

output_path = '../files/output/metrics.json'
os.makedirs(os.path.dirname(output_path), exist_ok=True) # Otra forma de crear/verificar la dirección

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(train_metrics, f, ensure_ascii=False) #indent=4
    f.write('\n')
    json.dump(test_metrics, f, ensure_ascii=False) #indent=4
    f.write('\n')